# XGBoost Multi-class Classification

XGBoost is a gradient boosting ensemble Machine Learning technique based on decision trees.
We've already seen what the terms decision tree, bagging and random forest mean in the Random Forest section, so let's take a closer look at what's new in XGBoost.

## What does boosting mean ?

Boosting is an ensemble strategy that adds new models to repair faults generated by current models. First, a model is constructed using the training data. The second model is then constructed in an attempt to address the faults in the previous model and models are added sequentially until no further advancements are possible. This ensemble method attempts to build a powerful classifier using previously 'weaker' classifiers.

## What does gradient means ?

Gradient boosting is so named because it employs a gradient descent approach to minimize loss when adding new models.

## What does eXtreme mean

XGBoost (eXtreme Gradient Boosting) is an advanced implementation of a gradient boosting algorithm.

## Advantages

-  XGBoost Execution Speed:
XGBoost is generally quick compared to other gradient boosting implementations.

- Performance: XGBoost has a strong track record of producing high-quality results in various machine learning tasks, especially in Kaggle competitions, where it has been a popular choice for winning solutions.

- Regularization :
Standard GBM implementation has no regularization like XGBoost; therefore, it also helps to reduce overfitting. In fact, XGBoost is also known as a ‘regularized boosting‘ technique.

- High Flexibility:
XGBoost allows users to define custom optimization objectives and evaluation criteria. This adds a whole new dimension to the model and there is no limit to what we can do.

- Handling Missing Values:
XGBoost has an in-built routine to handle missing values. The user is required to supply a different value than other observations and pass that as a parameter. XGBoost tries different things as it encounters a missing value on each node and learns which path to take for missing values in the future.

- Interpretability: Unlike some machine learning algorithms that can be difficult to interpret, XGBoost provides feature importances, allowing for a better understanding of which variables are most important in making predictions.

## Drawbacks

- Computational Complexity:
XGBoost can be computationally intensive, especially when training large models, making it less suitable for resource-constrained systems.

- Overfitting: XGBoost can be prone to overfitting, especially when trained on small datasets or when too many trees are used in the model

- Hyperparameter Tuning: XGBoost has many hyperparameters that can be adjusted, making it important to properly tune the parameters to optimize performance. However, finding the optimal set of parameters can be time-consuming and requires expertise.

- Memory Requirements: XGBoost can be memory-intensive, especially when working with large datasets, making it less suitable for systems with limited memory resources.

## Tune Hyperparameters

### General Parameters

The general parameter are here to access overall functionalities

- booster Select the type of model to run at each iteration. It has 2 options:
gbtree: tree-based models
gblinear: linear models

We will use in this project tree-based models due to its performance

### Booster Parameters for Tree Booster

eta [default=0.3]

    Analogous to the learning rate in GBM
    Makes the model more robust by shrinking the weights on each step
    Typical final values to be used: 0.01-0.2


    

### Learning Task Parameters
